In [13]:
import keras 
from keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

train_data.shape, train_targets.shape

((404, 13), (404,))

In [4]:
HIDDEN_SIZES = [ 20, 20 ]

In [5]:
from sklearn.metrics import mean_squared_error
def cross_val_score(X, y, build_model, cv=10):
    X = np.array(X)
    y = np.array(y)
    part = len(X) // cv
    results = np.empty(cv)
    for i in range(cv):
        train_X = np.concatenate(
            (X[:i*part],
            X[((i+1)*part):])
        )
        train_y = np.concatenate(
            (y[:i*part],
            y[(i+1)*part:])
        )
        test_X = X[i*part:(i+1)*part]
        test_y = y[i*part:(i+1)*part]
        
        model = build_model()
        
        model.fit(train_X, train_y, epochs=40, batch_size=16, verbose=0)
        predicted = model.predict(test_X)
        
        results[i] = mean_squared_error(test_y, predicted)
        
        print('score %d: %f' % (i + 1, results[i]))
        
    return results.mean()

# Sequential API

In [6]:
def build_seq_model(input_shape=13):
    model = keras.models.Sequential()
    for index, size in enumerate(HIDDEN_SIZES):
        if index == 0:
            model.add(layers.Dense(size, activation='relu', input_shape=(input_shape,)))
        else:
            model.add(layers.Dense(size, activation='relu'))
    model.add(layers.Dense(1))
    model.compile('rmsprop', loss='mse')
    
    return model

In [107]:
model = build_seq_model()
model.fit(train_data, train_targets, epochs=1200, batch_size=32, verbose=0)

Epoch 1/120
404/404 [==============================] - 0s 438us/step - loss: 229.8663
Epoch 2/120
404/404 [==============================] - 0s 57us/step - loss: 118.8035
Epoch 3/120
404/404 [==============================] - 0s 94us/step - loss: 88.7734
Epoch 4/120
404/404 [==============================] - 0s 86us/step - loss: 80.6045
Epoch 5/120
404/404 [==============================] - 0s 86us/step - loss: 72.5423
Epoch 6/120
404/404 [==============================] - 0s 81us/step - loss: 68.5115
Epoch 7/120
404/404 [==============================] - 0s 57us/step - loss: 71.9424
Epoch 8/120
404/404 [==============================] - 0s 89us/step - loss: 65.1539
Epoch 9/120
404/404 [==============================] - 0s 64us/step - loss: 64.6917
Epoch 10/120
404/404 [==============================] - 0s 74us/step - loss: 66.0602
Epoch 11/120
404/404 [==============================] - 0s 47us/step - loss: 61.2504
Epoch 12/120
404/404 [==============================] - 0s 42us/step - 

In [166]:
%%time
cross_val_score(train_data, train_targets, build_seq_model)

score 1: 54.909857
score 2: 14.191667
score 3: 51.907463
score 4: 38.013320
score 5: 45.592122
score 6: 27.559410
score 7: 30.176370
score 8: 74.554982
score 9: 108.809416
score 10: 28.408072
Wall time: 1min 42s


47.41226779200325

In [8]:
# linear predictions

from sklearn import model_selection
from sklearn.linear_model import LinearRegression
-model_selection.cross_val_score(LinearRegression(), train_data, train_targets, cv=10, scoring='neg_mean_squared_error').mean()

25.075378478863474

# Functional API

In [9]:
from keras.layers import Input, Dense
from keras.models import Model

def build_func_model(input_shape=13):
    inputs = x = Input(shape=(13,))
    
    for size in HIDDEN_SIZES:
        x = Dense(size, activation='relu')(x)
        
    predictions = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile('rmsprop', loss='mse')
    
    return model

In [170]:
%%time
cross_val_score(train_data, train_targets, build_func_model)

score 1: 64.513101
score 2: 15.015668
score 3: 52.995267
score 4: 42.571544
score 5: 27.660728
score 6: 48.976431
score 7: 21.550745
score 8: 98.399916
score 9: 76.177416
score 10: 31.514238
Wall time: 1min 43s


47.93750529987219

# Subclassing API

In [10]:
class SubModel(Model):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(20, activation='relu')
        self.dense2 = Dense(20, activation='relu')
        self.predictions =  Dense(1)
        self.compile('rmsprop', loss='mse')
        
    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        
        return self.predictions(x)

def build_subclassing_model():
    return SubModel()

In [12]:
%%time
cross_val_score(train_data, train_targets, build_subclassing_model)

score 1: 57.393706
score 2: 13.561560
score 3: 41.599184
score 4: 46.019086
score 5: 36.840373
score 6: 22.568422
score 7: 22.852063
score 8: 105.411047
score 9: 91.451602
score 10: 32.985760
Wall time: 20.1 s


47.06828030310907